## Load Packages

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Load Data

In [2]:
train = pd.read_csv("train.csv", index_col = "id")
test = pd.read_csv("test.csv", index_col = "id")

In [3]:
train.describe()

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
count,86.000000,86.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,86.000000,87.000000
mean,1264.244186,290.383721,982.570115,479.340920,494.904023,212.732874,118.780920,1325.096437,40.218391,0.482759,2066.534884,0.333333
std,765.452376,490.283499,617.332545,344.326452,311.836604,173.553264,96.218344,791.602538,10.461919,0.502599,1198.401364,0.474137
min,112.000000,30.000000,74.400000,36.610000,39.590000,0.000000,4.200000,209.250000,19.000000,0.000000,72.000000,0.000000
25%,685.500000,77.500000,549.390000,237.920000,272.745000,78.815000,52.425000,780.615000,33.000000,0.000000,1461.250000,0.000000
50%,1108.500000,124.500000,871.710000,423.270000,459.720000,188.780000,89.790000,1179.270000,41.000000,0.000000,1757.500000,0.000000
75%,1602.250000,244.500000,1268.085000,624.450000,624.360000,262.845000,155.450000,1617.725000,49.500000,1.000000,2238.250000,1.000000
max,4145.000000,3124.000000,3791.230000,2548.100000,1517.810000,878.040000,485.860000,4757.280000,60.000000,1.000000,7515.000000,1.000000


In [4]:
# drop missing value 
train.dropna(inplace =  True)

In [5]:
train.isna().any()

MO HLADR+ MFI (cells/ul)    False
Neu CD64+MFI (cells/ul)     False
CD3+T (cells/ul)            False
CD8+T (cells/ul)            False
CD4+T (cells/ul)            False
NK (cells/ul)               False
CD19+ (cells/ul)            False
CD45+ (cells/ul)            False
Age                         False
Sex 0M1F                    False
Mono CD64+MFI (cells/ul)    False
label                       False
dtype: bool

In [6]:
# label distribution
train.groupby(["label"]).size()

label
0    57
1    29
dtype: int64

In [7]:
# generate training data
x_train = train.drop(columns = ["label"])
y_train = train[["label"]]

## Random Forest Regression

In [8]:
# parameters tuning
best_param = [10, 1]
best_score = -1
for p1 in range(10, 100, 10):
    for p2 in range(1, 12):
        clf = RandomForestClassifier(n_estimators = p1, max_depth = p2, random_state=0)
        scores = cross_val_score(clf, x_train.values, y_train["label"], cv=5)
        s = scores.mean()
        if s > best_score:
            best_param = [p1, p2]
            best_score = s

In [9]:
best_param, best_score

([60, 4], 0.9424836601307189)

In [10]:
clf_rf = RandomForestClassifier(n_estimators = 60, max_depth = 4, random_state=0)
clf_rf.fit(x_train, y_train["label"])

RandomForestClassifier(max_depth=4, n_estimators=60, random_state=0)

In [11]:
test_rf = test.copy()
pred_rf = clf_rf.predict(test_rf)

In [12]:
output_rf = test_rf
output_rf["label"] = pred_rf

In [13]:
output_rf[["label"]].to_csv("results_rf.csv")

## Logistic Regression

In [14]:
clf_lr = LogisticRegression(random_state=0)

In [15]:
clf_lr.fit(x_train, y_train)

F:\Anaconda\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\Anaconda\envs\pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [16]:
clf_lr.score(x_train, y_train)

0.9302325581395349

In [17]:
test_lr = test.copy()
pred_lr = clf_lr.predict(test_lr)

In [18]:
output_lr = test_lr
output_lr["label"] = pred_lr

In [19]:
output_lr[["label"]].to_csv("results_lr.csv")

I submitted both of the result table above, it shows the logistic regression model shows better perfomance on the test dataset.